In [1]:
!pip install transformers

In [2]:
!pip install openai

In [3]:
# Create the directory structure
import os

dirs = [
    'Trust_me_Im_wrong',
    'Trust_me_Im_wrong/semantic_uncertainty',
    'Trust_me_Im_wrong/semantic_uncertainty/uncertainty',

    'Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models',
    'Trust_me_Im_wrong/semantic_uncertainty/uncertainty/uncertainty_measures',

]

for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

print("Directory structure created!")

Directory structure created!


In [4]:

import os

dirs = [
    'Trust_me_Im_wrong/analysis',
    'Trust_me_Im_wrong//visualization',
     'Trust_me_Im_wrong/reporting',
     'Trust_me_Im_wrong/reports',
     'Trust_me_Im_wrong/visualizations'
]
for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

In [5]:
# Create __init__.py in each directory to mark as package
for dir_path in dirs:
    init_path = os.path.join(dir_path, '__init__.py')
    if not os.path.exists(init_path):
        with open(init_path, 'w', encoding='utf-8') as f:
            f.write('# Init file for package\n')
        print(f'Created __init__.py in {dir_path}')
    else:
        print(f'__init__.py already exists in {dir_path}')

Created __init__.py in Trust_me_Im_wrong/analysis
Created __init__.py in Trust_me_Im_wrong//visualization
Created __init__.py in Trust_me_Im_wrong/reporting
Created __init__.py in Trust_me_Im_wrong/reports
Created __init__.py in Trust_me_Im_wrong/visualizations


In [6]:
%%writefile /content/Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models/base_model.py
from abc import ABC, abstractmethod
from typing import List, Text, Optional, Tuple

# Full stop sequences for post-processing (all languages)
FULL_STOP_SEQUENCES = [
    '\n',
]

# API-compatible stop sequences (max 4 for DeepSeek/OpenAI API)
API_STOP_SEQUENCES = [
    '\n',       # Double newline - most common separator
    '.',        # Period - ends sentences
]

# Keep original for backward compatibility
STOP_SEQUENCES = FULL_STOP_SEQUENCES


class BaseModel(ABC):
    """Base model class with enhanced stop sequence handling."""

    # Class variables
    stop_sequences: List[Text] = FULL_STOP_SEQUENCES
    api_stop_sequences: List[Text] = API_STOP_SEQUENCES

    def __init__(self):
        """Initialize with both API and full stop sequences."""
        self.stop_sequences = FULL_STOP_SEQUENCES
        self.api_stop_sequences = API_STOP_SEQUENCES

    @abstractmethod
    def predict(self, input_data: str, temperature: float):
        """
        Generate a response from the model given input_data and temperature.
        """
        pass

    @abstractmethod
    def get_p_true(self, input_data: str):
        """
        Compute probability that the answer to input_data is 'True'.
        """
        pass

    @staticmethod
    def post_process_with_stops(
        text: str,
        stop_sequences: Optional[List[str]] = None,
        preserve_stop: bool = False
    ) -> str:
        """
        Post-process text by truncating at the first occurrence of any stop sequence.

        Args:
            text: Input text to process
            stop_sequences: List of stop sequences (uses FULL_STOP_SEQUENCES if None)
            preserve_stop: If True, include the stop sequence in output

        Returns:
            Truncated text
        """
        if not text:
            return text

        if stop_sequences is None:
            stop_sequences = FULL_STOP_SEQUENCES

        # Find the earliest occurrence of any stop sequence
        earliest_pos = len(text)
        earliest_stop = None

        for stop in stop_sequences:
            pos = text.find(stop)
            if pos != -1 and pos < earliest_pos:
                earliest_pos = pos
                earliest_stop = stop

        # Truncate at the earliest stop sequence
        if earliest_pos < len(text):
            if preserve_stop and earliest_stop:
                return text[:earliest_pos + len(earliest_stop)]
            else:
                return text[:earliest_pos]

        return text

    @staticmethod
    def clean_for_comparison(text: str) -> str:
        """
        Aggressively clean text for accurate comparison and hallucination detection.
        Removes special characters that shouldn't be considered as tokens.

        Args:
            text: Text to clean

        Returns:
            Cleaned text
        """
        if not text:
            return ""

        # Convert to lowercase
        text = text.lower()

        # Remove common articles and fillers
        remove_words = ['the', 'a', 'an', 'is', 'are', 'was', 'were']
        for word in remove_words:
            text = text.replace(f" {word} ", " ")

        # Remove all punctuation and special characters
        special_chars = [
             '!', '?', ';', ',', '.', ':', '"', "'", "-", "_"
        ]

        for char in special_chars:
            text = text.replace(char, ' ')

        # Normalize whitespace
        text = ' '.join(text.split())

        return text.strip()

    @staticmethod
    def get_api_compatible_stops() -> List[str]:
        """
        Get API-compatible stop sequences (max 4).

        Returns:
            List of up to 4 stop sequences for API calls
        """
        return API_STOP_SEQUENCES[:4]

    @staticmethod
    def get_all_stops() -> List[str]:
        """
        Get all stop sequences for post-processing.

        Returns:
            Complete list of stop sequences
        """
        return FULL_STOP_SEQUENCES

Writing /content/Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models/base_model.py


In [7]:
%%writefile /content/Trust_me_Im_wrong/calc_semantic_entropy_api.py
"""
Semantic Entropy Calculation for API Models (DeepSeek V3 - English) - CORRECTED
Uses DeepSeek API with logprobs and sentence transformers for clustering.

FIXES APPLIED:
1. Softmax-based probability estimation (fixes logprob=0 issue)
2. Corrected entropy calculations with proper clipping
3. Sequence-based logprob extraction
"""

import json
import logging
import random
from collections import defaultdict
import numpy as np
from openai import OpenAI
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

# Try to import google colab userdata, fallback to env vars
try:
    from google.colab import userdata
    def get_secret(key):
        return userdata.get(key)
except ImportError:
    import os
    def get_secret(key):
        return os.environ.get(key)


# ============================================================================
# STOP SEQUENCES
# ============================================================================
FULL_STOP_SEQUENCES = ['\n']
API_STOP_SEQUENCES = ['\n', '.']


# ============================================================================
# HELPER: Post-process text with stop sequences
# ============================================================================
def post_process_with_stops(text, stop_sequences=None, preserve_stop=False):
    """Truncate text at the first occurrence of any stop sequence."""
    if not text:
        return text
    if stop_sequences is None:
        stop_sequences = FULL_STOP_SEQUENCES

    earliest_pos = len(text)
    earliest_stop = None

    for stop in stop_sequences:
        pos = text.find(stop)
        if pos != -1 and pos < earliest_pos:
            earliest_pos = pos
            earliest_stop = stop

    if earliest_pos < len(text):
        if preserve_stop and earliest_stop:
            return text[:earliest_pos + len(earliest_stop)]
        else:
            return text[:earliest_pos]
    return text


# ============================================================================
# CORRECTED: Softmax Normalizer for LogProbs
# ============================================================================
def softmax_normalize(logprobs):
    """
    Apply softmax normalization to logprobs for numerical stability.

    Args:
        logprobs: List of log probabilities

    Returns:
        List of normalized probabilities that sum to ~1.0
    """
    if not logprobs:
        return []

    # Convert to numpy for stability
    lps = np.array(logprobs, dtype=np.float64)

    # Clip extreme values
    lps = np.clip(lps, -100, 0)

    # Subtract max for numerical stability (prevents overflow)
    max_lp = np.max(lps)
    exp_scores = np.exp(lps - max_lp)

    # Normalize
    sum_exp = np.sum(exp_scores)
    if sum_exp == 0:
        return [1.0 / len(logprobs)] * len(logprobs)  # Uniform fallback

    probs = exp_scores / sum_exp
    return probs.tolist()


def estimate_logprob_from_alternatives(top_logprobs, debug=False):
    """
    Estimate meaningful logprob when API returns 0 for selected token.

    Uses softmax over alternatives to get relative probability.

    Args:
        top_logprobs: List of top_logprobs objects from API
        debug: Print debug info

    Returns:
        Estimated logprob (negative value)
    """
    if not top_logprobs or len(top_logprobs) < 2:
        return -1.0  # Default uncertainty

    raw_lps = []
    for alt in top_logprobs[:10]:
        lp = getattr(alt, 'logprob', None)
        if lp is not None:
            raw_lps.append(lp)

    if len(raw_lps) < 2:
        return -1.0

    # Apply softmax to get probabilities
    probs = softmax_normalize(raw_lps)

    # First token's estimated probability
    estimated_prob = probs[0] if probs else 0.5

    # Convert back to logprob
    estimated_logprob = np.log(estimated_prob + 1e-10)

    if debug:
        print(f"  Estimated logprob: {estimated_logprob:.4f} (prob={estimated_prob:.4f})")

    return float(estimated_logprob)


# ============================================================================
# Semantic Entropy Calculator
# ============================================================================
class SemanticEntropyAPI:
    """
    Semantic entropy calculation with CORRECTED logprobs handling.

    Key Fix: When DeepSeek returns logprob=0 for selected token,
    we estimate using softmax over alternatives.
    """

    def __init__(self, model_name="deepseek-chat", dataset_path="datasets/",
                 entailment_model="sentence_transformer", max_new_tokens=10, debug=False):
        """
        Initialize semantic entropy calculator for DeepSeek API.
        """
        random.seed(0)
        self.model_name = model_name
        self.max_new_tokens = max_new_tokens
        self.debug = debug

        # 1. Initialize API client for DeepSeek
        try:
            api_key = get_secret('deepseek')
            if not api_key:
                raise ValueError("DeepSeek API key not found in secrets.")

            self.client = OpenAI(
                api_key=api_key,
                base_url="https://api.deepseek.com"
            )
        except Exception as e:
            raise ValueError(f"Failed to initialize DeepSeek API client: {e}")

        # 2. Initialize Tokenizer
        try:
            hf_token = get_secret('hftoken')
            if not hf_token:
                print("Warning: 'hftoken' secret not found. Tokenizer load might fail if model is gated.")

            tokenizer_path = "deepseek-ai/DeepSeek-V3"
            print(f"Loading DeepSeek tokenizer from {tokenizer_path}...")

            self.tokenizer = AutoTokenizer.from_pretrained(
                tokenizer_path,
                token=hf_token,
                trust_remote_code=True
            )
        except Exception as e:
            raise RuntimeError(f"CRITICAL: Failed to load DeepSeek tokenizer. Error: {e}")

        # 3. Initialize sentence transformer for semantic similarity
        # KEEPING English model as requested
        print("Loading English embedding model for semantic clustering...")
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

        # Clustering threshold
        self.clustering_threshold = 0.5

        # Setup stop sequences
        self.api_stops = API_STOP_SEQUENCES[:4]
        self.full_stops = FULL_STOP_SEQUENCES

        print(f"Initialized SemanticEntropyAPI with CORRECTED logprobs handling")
        print(f"  Model: {model_name}")
        print(f"  Clustering threshold: {self.clustering_threshold}")

    def generate_answers(self, prompt, answer, num_generations=11, temperature=1.0, compute_acc=False):
        """
        Generate multiple answers for entropy calculation.

        Args:
            prompt: Input prompt
            answer: Ground truth answer
            num_generations: Number of generations (default 11)
            temperature: Sampling temperature
            compute_acc: Whether to compute accuracy

        Returns:
            Dictionary with generations and metadata
        """
        generations = {prompt: {"question": prompt}}
        full_responses = []
        all_generation_texts = []

        system_message = "Provide direct, brief answers without explanation."

        print(f"Generating {num_generations} responses...")

        for i in range(num_generations):
            # First generation at low temperature for "most likely" answer
            temp = 0.1 if i == 0 else temperature

            response_data = self._get_api_response(prompt, temp, system_message)
            all_generation_texts.append(response_data["text"])

            if i == 0:
                most_likely_answer_dict = {
                    "response": response_data["text"],
                    "token_log_likelihoods": response_data["token_logprobs"],
                    "embedding": None,
                    "accuracy": self._check_accuracy(response_data["text"], answer) if compute_acc else 0.0,
                    "total_logprob": response_data["total_logprob"]
                }
                generations[prompt]["most_likely_answer"] = most_likely_answer_dict
            else:
                full_responses.append((
                    response_data["text"],
                    response_data["token_logprobs"],
                    None,
                    self._check_accuracy(response_data["text"], answer) if compute_acc else 0.0,
                    response_data["total_logprob"]
                ))

        generations[prompt]["responses"] = full_responses
        generations[prompt]["reference"] = answer
        generations[prompt]["all_generation_texts"] = all_generation_texts

        print(f"Generated texts: {all_generation_texts[:3]}...")

        return {
            "accuracies": [most_likely_answer_dict["accuracy"]] if generations[prompt].get("most_likely_answer") else [],
            "generations": generations,
            "question": prompt,
            "reference": answer,
            "all_generation_texts": all_generation_texts
        }

    def _get_api_response(self, prompt, temperature, system_message=None):
        """
        Get response from DeepSeek API with CORRECTED logprobs extraction.

        Fix: When logprob=0 (broken), estimate using softmax over top_logprobs.
        """
        try:
            messages = []
            if system_message:
                messages.append({"role": "system", "content": system_message})
            messages.append({"role": "user", "content": prompt})

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                max_tokens=self.max_new_tokens,
                logprobs=True,
                top_logprobs=20,
                stop=self.api_stops
            )

            choice = response.choices[0]
            raw_text = choice.message.content or ""
            processed_text = post_process_with_stops(raw_text, self.full_stops)

            # CORRECTED: Extract logprobs with fallback estimation
            total_logprob = 0.0
            token_logprobs = []

            logprobs_content = None
            if hasattr(choice, 'logprobs') and choice.logprobs:
                logprobs_content = getattr(choice.logprobs, 'content', None)

            if logprobs_content:
                for token_data in logprobs_content:
                    logprob = getattr(token_data, 'logprob', None)

                    # CRITICAL FIX: If logprob is 0 or None, estimate from alternatives
                    if logprob is None or logprob >= -0.001:  # Effectively 0
                        top_lps = getattr(token_data, 'top_logprobs', [])
                        logprob = estimate_logprob_from_alternatives(top_lps, self.debug)

                    token_logprobs.append(logprob)
                    total_logprob += logprob

                    if self.debug:
                        token_text = getattr(token_data, 'token', '?')
                        print(f"  Token: '{token_text}', logprob: {logprob:.4f}")

            return {
                "text": processed_text,
                "raw_text": raw_text,
                "token_logprobs": token_logprobs,
                "total_logprob": total_logprob
            }

        except Exception as e:
            print(f"API error: {e}")
            return {
                "text": "",
                "raw_text": "",
                "token_logprobs": [],
                "total_logprob": -10.0
            }

    def _check_accuracy(self, generated, target):
        """Check if generated text matches target (basic check)."""
        if not generated or not target:
            return 0.0

        gen_clean = generated.lower().strip()
        target_clean = target.lower().strip()

        if target_clean in gen_clean or gen_clean in target_clean:
            return 1.0
        return 0.0

    def compute_uncertainty_measures(self, model_generations, compute_predictive_entropy=True, strict_entailment=False):
        """
        Compute semantic entropy and other uncertainty measures.

        Returns:
            Tuple of (avg_entropies dict, result_dict with debug info)
        """
        result_dict = {"semantic_ids": [], "debug_info": {}}
        entropies = defaultdict(list)

        for tid in model_generations:
            example = model_generations[tid]
            full_responses = example.get("responses", [])

            # Include the most likely answer
            if "most_likely_answer" in example:
                most_likely = example["most_likely_answer"]
                tll = most_likely.get("token_log_likelihoods", [])
                full_responses.insert(0, (
                    most_likely["response"],
                    tll,
                    None,
                    most_likely.get("accuracy", 0),
                    most_likely.get("total_logprob", sum(tll) if tll else -10)
                ))

            if not full_responses:
                continue

            print(f"\nProcessing {len(full_responses)} total generations")

            raw_responses = [r[0] for r in full_responses]
            responses = [post_process_with_stops(r, self.full_stops) for r in raw_responses]

            if compute_predictive_entropy:
                # Get total log likelihoods
                total_log_liks = []
                for r in full_responses:
                    if len(r) >= 5 and r[4] is not None:
                        total_log_liks.append(r[4])
                    elif r[1]:
                        total_log_liks.append(sum(r[1]))
                    else:
                        total_log_liks.append(-10)

                print(f"Total log likelihoods: {[f'{ll:.2f}' for ll in total_log_liks[:3]]}...")

                # Compute semantic IDs using clustering
                semantic_ids = self._get_semantic_ids_clustering(responses)
                result_dict["semantic_ids"].append(semantic_ids)

                print(f"Semantic IDs: {semantic_ids}")
                unique_clusters = len(set(sid for sid in semantic_ids if sid >= 0))
                print(f"Number of semantic clusters: {unique_clusters}")

                result_dict["debug_info"]["num_generations"] = len(full_responses)
                result_dict["debug_info"]["num_clusters"] = unique_clusters
                result_dict["debug_info"]["responses_sample"] = responses[:3]

                # Cluster assignment entropy (CLIPPED)
                cluster_entropy = self._cluster_assignment_entropy(semantic_ids)
                cluster_entropy = max(0.0, cluster_entropy)  # FIX: Clip to non-negative
                entropies["cluster_assignment_entropy"].append(cluster_entropy)
                print(f"Cluster assignment entropy: {cluster_entropy:.4f}")

                # Regular entropy
                regular_entropy = self._predictive_entropy(total_log_liks)
                regular_entropy = max(0.0, regular_entropy)  # FIX: Clip
                entropies["regular_entropy"].append(regular_entropy)
                print(f"Regular entropy: {regular_entropy:.4f}")

                # Semantic entropy
                log_lik_per_cluster = self._logsumexp_by_id(semantic_ids, total_log_liks)
                semantic_entropy = self._semantic_entropy(log_lik_per_cluster)
                semantic_entropy = max(0.0, semantic_entropy)  # FIX: Clip
                entropies["semantic_entropy"].append(semantic_entropy)
                print(f"Semantic entropy: {semantic_entropy:.4f}")

        avg_entropies = {k: float(np.mean(v)) if v else 0.0 for k, v in entropies.items()}
        return avg_entropies, result_dict

    def _get_semantic_ids_clustering(self, responses):
        """Cluster responses using sentence embeddings."""
        if not responses or len(responses) <= 1:
            return [0] * len(responses)

        valid_responses = [r for r in responses if r and r.strip()]
        if not valid_responses:
            return [0] * len(responses)

        print(f"Clustering {len(valid_responses)} responses")

        try:
            embeddings = self.embedding_model.encode(valid_responses)
        except Exception as e:
            print(f"Embedding error: {e}")
            return [0] * len(responses)

        if len(valid_responses) == 1:
            return [0] * len(responses)

        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=self.clustering_threshold,
            linkage='average'
        )

        try:
            semantic_ids = clustering.fit_predict(embeddings)

            # Map back to full response list
            full_ids = []
            valid_idx = 0
            for r in responses:
                if r and r.strip():
                    full_ids.append(int(semantic_ids[valid_idx]))
                    valid_idx += 1
                else:
                    full_ids.append(-1)
            return full_ids
        except Exception as e:
            print(f"Clustering error: {e}")
            return [0] * len(responses)

    def _cluster_assignment_entropy(self, semantic_ids):
        """Calculate entropy of cluster assignments."""
        if not semantic_ids:
            return 0.0

        valid_ids = [sid for sid in semantic_ids if sid >= 0]
        if not valid_ids:
            return 0.0

        counts = defaultdict(int)
        for sid in valid_ids:
            counts[sid] += 1

        total = len(valid_ids)
        entropy = 0.0
        for count in counts.values():
            p = count / total
            if p > 0:
                entropy -= p * np.log(p)

        return float(entropy)

    def _predictive_entropy(self, log_likelihoods):
        """Calculate predictive entropy from log likelihoods."""
        if not log_likelihoods:
            return 0.0

        # Normalize to get probabilities via softmax
        log_liks = np.array(log_likelihoods, dtype=np.float64)
        max_ll = np.max(log_liks)

        exp_liks = np.exp(log_liks - max_ll)
        probs = exp_liks / np.sum(exp_liks)

        # Calculate entropy
        entropy = 0.0
        for p in probs:
            if p > 1e-10:
                entropy -= p * np.log(p)

        return float(entropy)

    def _logsumexp_by_id(self, semantic_ids, log_likelihoods):
        """Aggregate log likelihoods by semantic cluster using logsumexp."""
        if len(semantic_ids) != len(log_likelihoods):
            return log_likelihoods

        clusters = defaultdict(list)
        for sid, ll in zip(semantic_ids, log_likelihoods):
            if sid >= 0:
                clusters[sid].append(ll)

        result = []
        for sid in sorted(clusters.keys()):
            lls = clusters[sid]
            # logsumexp for numerical stability
            max_ll = max(lls)
            aggregated = max_ll + np.log(sum(np.exp(ll - max_ll) for ll in lls))
            result.append(aggregated)

        return result

    def _semantic_entropy(self, log_likelihoods):
        """Calculate semantic entropy from aggregated log likelihoods."""
        return self._predictive_entropy(log_likelihoods)

    def calc_semantic_entropy_per_example(self, prompt, answer, temp=1.0, num_generations=11):
        """
        Calculate semantic entropy for a single example.

        Returns:
            Tuple of (entropy_dict, generation_details)
        """
        print(f"\nCalculating semantic entropy with {num_generations} generations (temp={temp})...")

        results = self.generate_answers(
            prompt=prompt,
            answer=answer,
            num_generations=num_generations,
            temperature=temp,
            compute_acc=True
        )

        avg_entropies, extra_info = self.compute_uncertainty_measures(
            results["generations"],
            compute_predictive_entropy=True,
            strict_entailment=False
        )

        # Add all generation texts to output
        avg_entropies["all_generations"] = results.get("all_generation_texts", [])
        avg_entropies["debug_info"] = extra_info.get("debug_info", {})

        return avg_entropies, results["generations"]


# ============================================================================
# Standalone Usage
# ============================================================================
if __name__ == "__main__":
    print("=" * 80)
    print("Semantic Entropy Calculator - CORRECTED VERSION")
    print("Fixes: logprob=0 estimation, entropy clipping")
    print("=" * 80)

    # Example usage:
    # calculator = SemanticEntropyAPI(model_name="deepseek-chat", debug=True)
    # result, details = calculator.calc_semantic_entropy_per_example(
    #     prompt="What is the capital of France?",
    #     answer="Paris",
    #     num_generations=11
    # )
    # print(f"Semantic entropy: {result['semantic_entropy']}")

Writing /content/Trust_me_Im_wrong/calc_semantic_entropy_api.py


In [8]:
%%writefile /content/Trust_me_Im_wrong/uncertainty_calculation_api.py
"""
Corrected Uncertainty Calculation API for DeepSeek (English) - V2
================================================================================

FIXES APPLIED:
1. LogProbsExtractor: Softmax-based probability estimation (fixes logprob=0)
2. EnglishTextMatcher: Fuzzy matching with number normalization
3. Entropy clipping to non-negative values
4. Proper semantic equivalence handling

Key Fix: DeepSeek API returns logprob=0 for selected token, making direct
probability extraction useless (exp(0)=1.0 always). This version uses softmax
normalization over all available top_logprobs to estimate meaningful probabilities.
"""

import json
import logging
import os
import random
import string
from collections import defaultdict
from difflib import SequenceMatcher

import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm
from transformers import AutoTokenizer

# Try to import google colab userdata, fallback to env vars
try:
    from google.colab import userdata
    def get_secret(key):
        return userdata.get(key)
except ImportError:
    def get_secret(key):
        return os.environ.get(key)


# ============================================================================
# STOP SEQUENCES
# ============================================================================
FULL_STOP_SEQUENCES = ['\n']
API_STOP_SEQUENCES = ['\n', '.']

# Tokens to exclude from alternatives
EXCLUDED_TOKENS = {
    '', ' ', '\n', '\t', '\r', '\\n', '\\t', '\\r',
    '<|endoftext|>', '<|im_end|>', '<|im_start|>',
    '<eos>', '<pad>', '<unk>', '<s>', '</s>',
    '▁', '##', '@@'
}


# ============================================================================
# HELPER: Post-process text with stop sequences
# ============================================================================
def post_process_with_stops(text, stop_sequences=None, preserve_stop=False):
    """Truncate text at the first occurrence of any stop sequence."""
    if not text:
        return text
    if stop_sequences is None:
        stop_sequences = FULL_STOP_SEQUENCES

    earliest_pos = len(text)
    earliest_stop = None

    for stop in stop_sequences:
        pos = text.find(stop)
        if pos != -1 and pos < earliest_pos:
            earliest_pos = pos
            earliest_stop = stop

    if earliest_pos < len(text):
        if preserve_stop and earliest_stop:
            return text[:earliest_pos + len(earliest_stop)]
        else:
            return text[:earliest_pos]
    return text


# ============================================================================
# CORRECTED: LogProbs Extractor with Softmax Estimation
# ============================================================================
class LogProbsExtractor:
    """
    Extracts and estimates probabilities from DeepSeek API responses.

    PROBLEM: DeepSeek API returns logprob=0 for the selected token, which
    gives exp(0)=1.0, making probability extraction useless.

    SOLUTION: Use softmax normalization over all available top_logprobs
    to estimate relative probabilities among the top candidates.

    Example:
        Raw logprobs: [0.0, -2.1, -4.5, -3.8]  (first is broken)
        After softmax: [0.857, 0.105, 0.009, 0.019]  (meaningful!)
    """

    def __init__(self, debug=False):
        self.debug = debug

    def _is_valid_token(self, raw_token):
        """
        Check if token should be included in probability calculation.

        Filters out:
        - Empty tokens
        - Special tokens (various Unicode formats)
        - Pure punctuation
        - Whitespace-only tokens
        - Sentence piece markers
        """
        if not raw_token:
            return False

        token = raw_token.strip()

        if not token:
            return False

        # Skip excluded tokens
        if raw_token in EXCLUDED_TOKENS or token in EXCLUDED_TOKENS:
            return False

        # ============================================================
        # ROBUST SPECIAL TOKEN DETECTION
        # DeepSeek uses various Unicode characters for special tokens
        # ============================================================

        # Standard ASCII special tokens: <|...|>
        if raw_token.startswith("<|") and raw_token.endswith("|>"):
            return False

        # Unicode fullwidth special tokens: <｜...｜>
        # ｜ is U+FF5C (fullwidth vertical line)
        if raw_token.startswith("<｜") or raw_token.endswith("｜>"):
            return False

        # Any token containing special markers
        special_markers = [
            "▁",           # Sentence piece marker (U+2581)
            "｜",           # Fullwidth vertical line (U+FF5C)
            "<|",          # ASCII special token start
            "|>",          # ASCII special token end
            "end_of",      # Common special token pattern
            "end▁of",      # With sentence piece
            "begin_of",    # Common special token pattern
            "begin▁of",    # With sentence piece
            "sentence",    # Part of special tokens
            "padding",     # Padding token
            "endoftext",   # End of text
            "im_start",    # Chat template
            "im_end",      # Chat template
            "<eos>",       # End of sequence
            "<bos>",       # Begin of sequence
            "<pad>",       # Padding
            "<unk>",       # Unknown
            "<s>",         # Start
            "</s>",        # End
        ]

        token_lower = raw_token.lower()
        for marker in special_markers:
            if marker.lower() in token_lower:
                return False

        # Skip tokens that are mostly special characters
        alnum_count = sum(1 for c in token if c.isalnum())
        if len(token) > 0 and alnum_count / len(token) < 0.5:
            return False

        # Skip pure punctuation (but allow mixed like "don't")
        if all(char in string.punctuation for char in token):
            return False

        # Skip very short tokens that are just symbols
        if len(token) == 1 and not token.isalpha():
            return False

        return True

    def _softmax_normalize(self, logprobs):
        """
        Apply softmax normalization to logprobs for numerical stability.

        Args:
            logprobs: List of log probabilities

        Returns:
            List of normalized probabilities that sum to ~1.0
        """
        if not logprobs:
            return []

        # Convert to numpy for stability
        lps = np.array(logprobs, dtype=np.float64)

        # Clip extreme values to prevent numerical issues
        lps = np.clip(lps, -100, 0)

        # Subtract max for numerical stability (prevents overflow)
        max_lp = np.max(lps)
        exp_scores = np.exp(lps - max_lp)

        # Normalize
        sum_exp = np.sum(exp_scores)
        if sum_exp == 0:
            return [1.0 / len(logprobs)] * len(logprobs)  # Uniform fallback

        probs = exp_scores / sum_exp
        return probs.tolist()

    def extract_all_probabilities(self, logprobs_content, max_alternatives=5, answer_text=None):
        """
        Extract first token probability and alternatives using softmax.

        Args:
            logprobs_content: The logprobs.content from API response
            max_alternatives: Maximum number of alternatives to return
            answer_text: The actual generated answer (for sequence calculation)

        Returns:
            dict with probabilities and metadata
        """
        result = {
            "first_token_probability": 0.5,
            "first_token_logprob": None,
            "probability_source": "unknown",
            "top_word_alternatives": [],
            "prob_diff_top2": 0.0,
            "debug_info": {
                "raw_logprobs": [],
                "all_tokens": [],
                "all_raw_tokens": [],
                "filtered_count": 0,
                "softmax_applied": False,
                "sequence_logprobs": [],
                "sequence_tokens": []
            }
        }

        # Early exit if no data
        if not logprobs_content or len(logprobs_content) == 0:
            result["probability_source"] = "no_data"
            if self.debug:
                print("WARNING: No logprobs content available")
            return result

        # ============================================================
        # SEQUENCE-BASED PROBABILITY CALCULATION
        # ============================================================
        if answer_text and len(logprobs_content) >= 1:
            seq_result = self._calculate_sequence_probability(
                logprobs_content, answer_text, max_alternatives
            )
            if seq_result is not None:
                return seq_result

        # Fallback to first token method
        return self._extract_first_token_probabilities(logprobs_content, max_alternatives)

    def _calculate_sequence_probability(self, logprobs_content, answer_text, max_alternatives=5):
        """
        Calculate probability based on FULL token sequence.

        Since classification happens FIRST, the first token is ALWAYS part of
        the answer. We calculate probability even if logprob=0 (broken).
        """
        result = {
            "first_token_probability": 0.5,
            "first_token_logprob": None,
            "probability_source": "unknown",
            "top_word_alternatives": [],
            "prob_diff_top2": 0.0,
            "debug_info": {
                "sequence_tokens": [],
                "sequence_logprobs": [],
                "valid_logprobs": [],
                "avg_logprob": None
            }
        }

        if not logprobs_content or len(logprobs_content) == 0:
            return None  # Use fallback

        first_token_data = logprobs_content[0]
        first_token = getattr(first_token_data, 'token', '')

        if self.debug:
            print(f"\nDEBUG SEQUENCE: Answer='{answer_text}', First token='{first_token}'")

        # ============================================================
        # COLLECT ALL TOKEN LOGPROBS IN THE SEQUENCE
        # ============================================================
        sequence_tokens = []
        sequence_logprobs = []

        for i, token_data in enumerate(logprobs_content):
            token = getattr(token_data, 'token', '')
            logprob = getattr(token_data, 'logprob', None)

            sequence_tokens.append(token)
            sequence_logprobs.append(logprob if logprob is not None else 0.0)

            if self.debug:
                print(f"  Token {i+1}: '{token}' → logprob={logprob}")

        result["debug_info"]["sequence_tokens"] = sequence_tokens
        result["debug_info"]["sequence_logprobs"] = sequence_logprobs

        # ============================================================
        # GET ALTERNATIVES FROM FIRST TOKEN'S TOP_LOGPROBS
        # ============================================================
        top_logprobs = getattr(first_token_data, 'top_logprobs', None) or []

        all_alt_logprobs = []
        all_alt_tokens = []

        for alt in top_logprobs:
            alt_token = getattr(alt, 'token', '')
            alt_logprob = getattr(alt, 'logprob', None)

            if alt_logprob is None:
                continue

            all_alt_tokens.append(alt_token.strip())
            all_alt_logprobs.append(alt_logprob)

        if self.debug:
            print(f"DEBUG: All alternatives from API: {list(zip(all_alt_tokens[:5], all_alt_logprobs[:5]))}")

        # ============================================================
        # CALCULATE FIRST TOKEN PROBABILITY USING SOFTMAX
        # ============================================================
        if len(all_alt_logprobs) >= 2:
            # Apply softmax to get probabilities
            all_lps = np.array(all_alt_logprobs, dtype=np.float64)
            all_lps = np.clip(all_lps, -100, 0)

            max_lp = np.max(all_lps)
            exp_scores = np.exp(all_lps - max_lp)
            probs = exp_scores / np.sum(exp_scores)

            first_token_prob = float(probs[0])
            first_token_logprob = float(all_alt_logprobs[0])

            result["first_token_probability"] = first_token_prob
            result["first_token_logprob"] = first_token_logprob
            result["probability_source"] = "softmax_from_alternatives"

            if self.debug:
                print(f"DEBUG: Softmax probs: {[round(p, 4) for p in probs[:5]]}")
                print(f"DEBUG: First token prob (softmax): {first_token_prob:.4f}")

            # ============================================================
            # BUILD ALTERNATIVES LIST WITH SOFTMAX PROBS
            # ============================================================
            alternatives = []
            seen_tokens = set()

            for i, (token, prob) in enumerate(zip(all_alt_tokens, probs)):
                if not token or not self._is_valid_token(token):
                    continue

                if token.lower() in seen_tokens:
                    continue

                seen_tokens.add(token.lower())
                alternatives.append({
                    "token": token,
                    "prob": float(prob),
                    "logprob": float(all_alt_logprobs[i])
                })

                if len(alternatives) >= max_alternatives:
                    break

            result["top_word_alternatives"] = alternatives

            # ============================================================
            # CALCULATE PROB_DIFF
            # ============================================================
            if len(alternatives) >= 2:
                result["prob_diff_top2"] = float(
                    alternatives[0]["prob"] - alternatives[1]["prob"]
                )
            elif len(alternatives) == 1:
                result["prob_diff_top2"] = float(alternatives[0]["prob"])

            if self.debug:
                if len(alternatives) >= 2:
                    print(f"DEBUG: prob_diff = {alternatives[0]['prob']:.4f} - {alternatives[1]['prob']:.4f} = {result['prob_diff_top2']:.4f}")
                print(f"DEBUG: probability_source: {result['probability_source']}")

        else:
            # Not enough alternatives - use sequence average as fallback
            avg_logprob = sum(sequence_logprobs) / len(sequence_logprobs) if sequence_logprobs else 0.0
            result["first_token_probability"] = float(np.exp(np.clip(avg_logprob, -100, 0)))
            result["first_token_logprob"] = avg_logprob
            result["probability_source"] = "sequence_average_fallback"
            result["debug_info"]["avg_logprob"] = avg_logprob

            if self.debug:
                print(f"DEBUG: Fallback to sequence average: {avg_logprob:.4f}")

        return result

    def _extract_first_token_probabilities(self, logprobs_content, max_alternatives=5):
        """
        Extract first token probability using softmax (fallback method).
        """
        result = {
            "first_token_probability": 0.5,
            "first_token_logprob": None,
            "probability_source": "unknown",
            "top_word_alternatives": [],
            "prob_diff_top2": 0.0,
            "debug_info": {
                "raw_logprobs": [],
                "all_tokens": [],
                "filtered_count": 0,
                "softmax_applied": False
            }
        }

        if not logprobs_content or len(logprobs_content) == 0:
            result["probability_source"] = "no_data"
            return result

        first_token_data = logprobs_content[0]
        top_logprobs = getattr(first_token_data, 'top_logprobs', None) or []

        if not top_logprobs or len(top_logprobs) < 2:
            result["probability_source"] = "insufficient_alternatives"
            return result

        # Collect valid entries
        valid_entries = []
        for alt in top_logprobs:
            raw_token = getattr(alt, 'token', '')
            logprob = getattr(alt, 'logprob', None)

            if not self._is_valid_token(raw_token) or logprob is None:
                continue

            token = raw_token.strip()
            is_complete = len(token) >= 4 or (len(token) > 0 and token[0].isupper())

            valid_entries.append({
                "token": token,
                "logprob": logprob,
                "is_complete": is_complete
            })

        if len(valid_entries) < 1:
            result["probability_source"] = "no_valid_alternatives"
            return result

        # Apply softmax
        all_logprobs = [e["logprob"] for e in valid_entries]
        softmax_probs = self._softmax_normalize(all_logprobs)
        result["debug_info"]["softmax_applied"] = True

        for i, entry in enumerate(valid_entries):
            entry["prob"] = softmax_probs[i]

        valid_entries.sort(key=lambda x: (x["is_complete"], x["prob"]), reverse=True)

        first_entry = valid_entries[0]
        result["first_token_probability"] = float(first_entry["prob"])
        result["first_token_logprob"] = float(first_entry["logprob"])
        result["probability_source"] = "softmax_estimated"

        # Build alternatives
        seen_tokens = set()
        alternatives = []
        for entry in valid_entries:
            if entry["token"] not in seen_tokens:
                seen_tokens.add(entry["token"])
                alternatives.append({
                    "token": entry["token"],
                    "prob": float(entry["prob"]),
                    "logprob": float(entry["logprob"])
                })
                if len(alternatives) >= max_alternatives:
                    break

        result["top_word_alternatives"] = alternatives

        if len(alternatives) >= 2:
            result["prob_diff_top2"] = float(alternatives[0]["prob"] - alternatives[1]["prob"])

        return result


# ============================================================================
# English Text Matcher (Fuzzy Matching with Number Normalization)
# ============================================================================
class EnglishTextMatcher:
    """
    English-aware text matching for answer comparison.

    Features:
    - Number normalization ("4" ↔ "four")
    - Fuzzy string matching
    - Token overlap scoring
    - Semantic equivalence pairs
    """

    ENGLISH_STOP_WORDS = {
        'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been',
        'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
        'would', 'could', 'should', 'may', 'might', 'must', 'shall',
        'can', 'of', 'in', 'to', 'for', 'with', 'on', 'at', 'by',
        'from', 'as', 'it', 'its', 'this', 'that', 'these', 'those',
        'and', 'or', 'but', 'if', 'then', 'so', 'because', 'although'
    }

    # Number mappings (digit to word)
    NUMBER_TO_WORD = {
        '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
        '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine',
        '10': 'ten', '11': 'eleven', '12': 'twelve', '13': 'thirteen',
        '14': 'fourteen', '15': 'fifteen', '16': 'sixteen', '17': 'seventeen',
        '18': 'eighteen', '19': 'nineteen', '20': 'twenty',
        '30': 'thirty', '40': 'forty', '50': 'fifty', '60': 'sixty',
        '70': 'seventy', '80': 'eighty', '90': 'ninety', '100': 'hundred',
        '1000': 'thousand', '1st': 'first', '2nd': 'second', '3rd': 'third',
        '4th': 'fourth', '5th': 'fifth', '6th': 'sixth', '7th': 'seventh',
        '8th': 'eighth', '9th': 'ninth', '10th': 'tenth'
    }

    # Word forms set (for checking if already a word)
    WORD_FORMS = {
        'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine',
        'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen',
        'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty',
        'sixty', 'seventy', 'eighty', 'ninety', 'hundred', 'thousand',
        'first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh',
        'eighth', 'ninth', 'tenth'
    }

    # Semantic equivalences (lowercase)
    SEMANTIC_EQUIVALENCES = [
        # Professions
        ('painter', 'artist'),
        ('artist', 'painter'),
        # Places
        ('via appia', 'appian way'),
        ('appian way', 'via appia'),
        # Music
        ('eroica', 'third symphony'),
        ('third symphony', 'eroica'),
        ('eroica', 'the third'),
        ('the third', 'eroica'),
        # Sports teams (add more as needed)
        ('packers', 'green bay packers'),
        ('steelers', 'pittsburgh steelers'),
        # Common equivalences
        ('usa', 'united states'),
        ('united states', 'usa'),
        ('uk', 'united kingdom'),
        ('united kingdom', 'uk'),
    ]

    def __init__(self):
        # Build equivalence lookup
        self.equivalence_map = defaultdict(set)
        for a, b in self.SEMANTIC_EQUIVALENCES:
            self.equivalence_map[a].add(b)
            self.equivalence_map[b].add(a)

    def normalize_text(self, text):
        """Normalize English text for comparison."""
        if not text:
            return ""

        # Lowercase
        text = text.lower()

        # Remove punctuation
        for char in '.,;:!?"\'()-[]{}':
            text = text.replace(char, ' ')

        # Normalize whitespace
        text = ' '.join(text.split())
        return text.strip()

    def normalize_number(self, text):
        """
        Normalize numbers to word form for comparison.

        "4" → "four"
        "Four" → "four"
        "1st" → "first"
        """
        if not text:
            return ""

        text = text.lower().strip()

        # Direct digit/ordinal lookup
        if text in self.NUMBER_TO_WORD:
            return self.NUMBER_TO_WORD[text]

        # Already a word form - return as is
        if text in self.WORD_FORMS:
            return text

        return text

    def get_meaningful_tokens(self, text):
        """Extract meaningful tokens, excluding stop words."""
        normalized = self.normalize_text(text)
        tokens = normalized.split()
        return [t for t in tokens if t not in self.ENGLISH_STOP_WORDS and len(t) > 1]

    def check_semantic_equivalence(self, gen_norm, target_norm):
        """Check if two texts are semantically equivalent."""
        # Direct equivalence
        if gen_norm in self.equivalence_map:
            if target_norm in self.equivalence_map[gen_norm]:
                return True

        # Check if any token pair is equivalent
        gen_tokens = set(gen_norm.split())
        target_tokens = set(target_norm.split())

        for gt in gen_tokens:
            if gt in self.equivalence_map:
                for equiv in self.equivalence_map[gt]:
                    if equiv in target_tokens or equiv == target_norm:
                        return True

        return False

    def calculate_match_score(self, generated, target):
        """
        Calculate match score between generated and target answer.

        Returns:
            Tuple of (score: 0.0-1.0, match_type: str)
        """
        if not generated or not target:
            return 0.0, "no_match"

        gen_norm = self.normalize_text(generated)
        target_norm = self.normalize_text(target)

        # ============================================================
        # 1. EXACT MATCH
        # ============================================================
        if gen_norm == target_norm:
            return 1.0, "exact"

        # ============================================================
        # 2. NUMBER NORMALIZATION
        # ============================================================
        gen_num = self.normalize_number(gen_norm)
        target_num = self.normalize_number(target_norm)

        if gen_num == target_num and gen_num != gen_norm:
            return 1.0, "number_match"

        # ============================================================
        # 3. SEMANTIC EQUIVALENCE
        # ============================================================
        if self.check_semantic_equivalence(gen_norm, target_norm):
            return 0.95, "semantic_equivalence"

        # ============================================================
        # 4. CONTAINS CHECK
        # ============================================================
        if target_norm in gen_norm:
            return 0.95, "contains"

        if gen_norm in target_norm:
            return 0.90, "contained_in"

        # ============================================================
        # 5. TOKEN OVERLAP
        # ============================================================
        gen_tokens = set(self.get_meaningful_tokens(generated))
        target_tokens = set(self.get_meaningful_tokens(target))

        if target_tokens and gen_tokens:
            # Target is subset of generated
            if target_tokens.issubset(gen_tokens):
                return 0.9, "token_subset"

            # Calculate overlap
            overlap = len(target_tokens & gen_tokens)
            target_size = len(target_tokens)

            if target_size > 0:
                overlap_ratio = overlap / target_size

                if overlap_ratio >= 0.8:
                    return 0.85, "high_token_overlap"
                elif overlap_ratio >= 0.6:
                    return 0.7, "medium_token_overlap"
                elif overlap_ratio >= 0.4:
                    return 0.5, "low_token_overlap"

        # ============================================================
        # 6. FUZZY STRING MATCHING
        # ============================================================
        similarity = SequenceMatcher(None, gen_norm, target_norm).ratio()

        if similarity >= 0.85:
            return similarity * 0.95, "fuzzy_very_high"
        elif similarity >= 0.7:
            return similarity * 0.9, "fuzzy_high"
        elif similarity >= 0.5:
            return similarity * 0.7, "fuzzy_medium"

        # ============================================================
        # 7. PARTIAL MATCH (any significant token matches)
        # ============================================================
        if len(target_tokens) >= 1:
            for token in target_tokens:
                if len(token) >= 4 and token in gen_norm:
                    return 0.4, "partial_match"

        return 0.0, "no_match"


# ============================================================================
# Semantic Entropy Calculator (Integrated)
# ============================================================================
class SemanticEntropyAPI:
    """Semantic entropy calculation with corrected logprobs handling."""

    def __init__(self, client, tokenizer, model_name="deepseek-chat",
                 max_new_tokens=10, debug=False):
        self.client = client
        self.tokenizer = tokenizer
        self.model_name = model_name
        self.max_new_tokens = max_new_tokens
        self.debug = debug

        # KEEP English model as requested
        print("Loading English embedding model for semantic clustering...")
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

        self.clustering_threshold = 0.5
        self.api_stops = API_STOP_SEQUENCES[:4]
        self.full_stops = FULL_STOP_SEQUENCES

        print(f"Initialized SemanticEntropyAPI with corrected logprobs handling")

    def calc_semantic_entropy_per_example(self, prompt, answer, temp=1.0, num_generations=11):
        """Calculate semantic entropy for a single example."""
        print(f"\nCalculating semantic entropy with {num_generations} generations (temp={temp})...")

        gen_result = self.generate_answers(prompt, answer, num_generations, temp)
        all_generation_texts = gen_result.get('all_generation_texts', [])

        entropies, result_dict = self.compute_uncertainty_measures(gen_result['generations'])

        return {
            'semantic_entropy': entropies.get('semantic_entropy', 0),
            'regular_entropy': entropies.get('regular_entropy', 0),
            'cluster_assignment_entropy': entropies.get('cluster_assignment_entropy', 0),
            'all_generations': all_generation_texts,
            'debug_info': result_dict.get('debug_info', {})
        }, result_dict

    def generate_answers(self, prompt, answer, num_generations=11, temperature=1.0):
        """Generate multiple answers for entropy calculation."""
        generations = {prompt: {"question": prompt}}
        full_responses = []
        all_generation_texts = []

        system_message = "Provide direct, brief answers without explanation."

        print(f"Generating {num_generations} responses...")

        for i in range(num_generations):
            temp = 0.1 if i == 0 else temperature
            response_data = self._get_api_response(prompt, temp, system_message)
            all_generation_texts.append(response_data["text"])

            if i == 0:
                most_likely_answer_dict = {
                    "response": response_data["text"],
                    "token_log_likelihoods": response_data["token_logprobs"],
                    "embedding": None,
                    "accuracy": 0.0,
                    "total_logprob": response_data["total_logprob"]
                }
                generations[prompt]["most_likely_answer"] = most_likely_answer_dict
            else:
                full_responses.append((
                    response_data["text"],
                    response_data["token_logprobs"],
                    None,
                    0.0,
                    response_data["total_logprob"]
                ))

        generations[prompt]["responses"] = full_responses
        generations[prompt]["reference"] = answer
        generations[prompt]["all_generation_texts"] = all_generation_texts

        print(f"Generated texts: {all_generation_texts[:3]}...")

        return {
            "accuracies": [],
            "generations": generations,
            "question": prompt,
            "reference": answer,
            "all_generation_texts": all_generation_texts
        }

    def _get_api_response(self, prompt, temperature, system_message=None):
        """Get response from DeepSeek API with CORRECTED logprobs extraction."""
        try:
            messages = []
            if system_message:
                messages.append({"role": "system", "content": system_message})
            messages.append({"role": "user", "content": prompt})

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                max_tokens=self.max_new_tokens,
                logprobs=True,
                top_logprobs=20,
                stop=self.api_stops
            )

            choice = response.choices[0]
            raw_text = choice.message.content or ""
            processed_text = post_process_with_stops(raw_text, self.full_stops)

            # CORRECTED: Extract logprobs with fallback estimation
            total_logprob = 0.0
            token_logprobs = []

            logprobs_content = None
            if hasattr(choice, 'logprobs') and choice.logprobs:
                logprobs_content = getattr(choice.logprobs, 'content', None)

            if logprobs_content:
                for token_data in logprobs_content:
                    logprob = getattr(token_data, 'logprob', None)

                    # CRITICAL FIX: If logprob is 0 or None, estimate from alternatives
                    if logprob is None or logprob >= -0.001:
                        top_lps = getattr(token_data, 'top_logprobs', [])
                        logprob = self._estimate_logprob(top_lps)

                    token_logprobs.append(logprob)
                    total_logprob += logprob

            return {
                "text": processed_text,
                "raw_text": raw_text,
                "token_logprobs": token_logprobs,
                "total_logprob": total_logprob
            }

        except Exception as e:
            print(f"API error: {e}")
            return {
                "text": "",
                "raw_text": "",
                "token_logprobs": [],
                "total_logprob": -10.0
            }

    def _estimate_logprob(self, top_logprobs):
        """Estimate logprob when API returns 0 for selected token."""
        if not top_logprobs or len(top_logprobs) < 2:
            return -1.0

        raw_lps = [getattr(alt, 'logprob', -10) for alt in top_logprobs[:10]]
        raw_lps = [lp if lp is not None else -10 for lp in raw_lps]

        if len(raw_lps) < 2:
            return -1.0

        # Softmax to estimate probability
        lps = np.array(raw_lps, dtype=np.float64)
        lps = np.clip(lps, -100, 0)
        max_lp = np.max(lps)
        exp_scores = np.exp(lps - max_lp)
        probs = exp_scores / np.sum(exp_scores)

        # Convert back to logprob
        return float(np.log(probs[0] + 1e-10))

    def compute_uncertainty_measures(self, model_generations, compute_predictive_entropy=True):
        """Compute semantic entropy and other uncertainty measures."""
        result_dict = {"semantic_ids": [], "debug_info": {}}
        entropies = defaultdict(list)

        for tid in model_generations:
            example = model_generations[tid]
            full_responses = example.get("responses", [])

            if "most_likely_answer" in example:
                most_likely = example["most_likely_answer"]
                tll = most_likely.get("token_log_likelihoods", [])
                full_responses.insert(0, (
                    most_likely["response"],
                    tll,
                    None,
                    most_likely.get("accuracy", 0),
                    most_likely.get("total_logprob", sum(tll) if tll else -10)
                ))

            if not full_responses:
                continue

            print(f"\nProcessing {len(full_responses)} total generations")

            raw_responses = [r[0] for r in full_responses]
            responses = [post_process_with_stops(r, self.full_stops) for r in raw_responses]

            if compute_predictive_entropy:
                total_log_liks = []
                for r in full_responses:
                    if len(r) >= 5 and r[4] is not None:
                        total_log_liks.append(r[4])
                    elif r[1]:
                        total_log_liks.append(sum(r[1]))
                    else:
                        total_log_liks.append(-10)

                print(f"Total log likelihoods: {[f'{ll:.2f}' for ll in total_log_liks[:3]]}...")

                semantic_ids = self._get_semantic_ids_clustering(responses)
                result_dict["semantic_ids"].append(semantic_ids)

                print(f"Semantic IDs: {semantic_ids}")
                unique_clusters = len(set(sid for sid in semantic_ids if sid >= 0))
                print(f"Number of semantic clusters: {unique_clusters}")

                result_dict["debug_info"]["num_generations"] = len(full_responses)
                result_dict["debug_info"]["num_clusters"] = unique_clusters

                # CLIPPED entropy calculations
                cluster_entropy = max(0.0, self._cluster_assignment_entropy(semantic_ids))
                entropies["cluster_assignment_entropy"].append(cluster_entropy)
                print(f"Cluster assignment entropy: {cluster_entropy:.4f}")

                regular_entropy = max(0.0, self._predictive_entropy(total_log_liks))
                entropies["regular_entropy"].append(regular_entropy)
                print(f"Regular entropy: {regular_entropy:.4f}")

                log_lik_per_cluster = self._logsumexp_by_id(semantic_ids, total_log_liks)
                semantic_entropy = max(0.0, self._semantic_entropy(log_lik_per_cluster))
                entropies["semantic_entropy"].append(semantic_entropy)
                print(f"Semantic entropy: {semantic_entropy:.4f}")

        avg_entropies = {k: float(np.mean(v)) if v else 0.0 for k, v in entropies.items()}
        return avg_entropies, result_dict

    def _get_semantic_ids_clustering(self, responses):
        """Cluster responses using sentence embeddings."""
        if not responses or len(responses) <= 1:
            return [0] * len(responses)

        valid_responses = [r for r in responses if r and r.strip()]
        if not valid_responses:
            return [0] * len(responses)

        print(f"Clustering {len(valid_responses)} responses")

        try:
            embeddings = self.embedding_model.encode(valid_responses)
        except Exception as e:
            print(f"Embedding error: {e}")
            return [0] * len(responses)

        if len(valid_responses) == 1:
            return [0] * len(responses)

        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=self.clustering_threshold,
            linkage='average'
        )

        try:
            semantic_ids = clustering.fit_predict(embeddings)
            full_ids = []
            valid_idx = 0
            for r in responses:
                if r and r.strip():
                    full_ids.append(int(semantic_ids[valid_idx]))
                    valid_idx += 1
                else:
                    full_ids.append(-1)
            return full_ids
        except Exception as e:
            print(f"Clustering error: {e}")
            return [0] * len(responses)

    def _cluster_assignment_entropy(self, semantic_ids):
        """Calculate entropy of cluster assignments."""
        if not semantic_ids:
            return 0.0

        valid_ids = [sid for sid in semantic_ids if sid >= 0]
        if not valid_ids:
            return 0.0

        counts = defaultdict(int)
        for sid in valid_ids:
            counts[sid] += 1

        total = len(valid_ids)
        entropy = 0.0
        for count in counts.values():
            p = count / total
            if p > 0:
                entropy -= p * np.log(p)

        return float(entropy)

    def _predictive_entropy(self, log_likelihoods):
        """Calculate predictive entropy from log likelihoods."""
        if not log_likelihoods:
            return 0.0

        log_liks = np.array(log_likelihoods, dtype=np.float64)
        max_ll = np.max(log_liks)

        exp_liks = np.exp(log_liks - max_ll)
        probs = exp_liks / np.sum(exp_liks)

        entropy = 0.0
        for p in probs:
            if p > 1e-10:
                entropy -= p * np.log(p)

        return float(entropy)

    def _logsumexp_by_id(self, semantic_ids, log_likelihoods):
        """Aggregate log likelihoods by semantic cluster using logsumexp."""
        if len(semantic_ids) != len(log_likelihoods):
            return log_likelihoods

        clusters = defaultdict(list)
        for sid, ll in zip(semantic_ids, log_likelihoods):
            if sid >= 0:
                clusters[sid].append(ll)

        result = []
        for sid in sorted(clusters.keys()):
            lls = clusters[sid]
            max_ll = max(lls)
            aggregated = max_ll + np.log(sum(np.exp(ll - max_ll) for ll in lls))
            result.append(aggregated)

        return result

    def _semantic_entropy(self, log_likelihoods):
        """Calculate semantic entropy from aggregated log likelihoods."""
        return self._predictive_entropy(log_likelihoods)


# ============================================================================
# MAIN: Uncertainty Calculation API
# ============================================================================
class UncertaintyCalculationAPI:
    """
    Main class for uncertainty calculation with:
    - SOFTMAX-based probability estimation
    - English fuzzy text matching
    - Corrected entropy calculations
    """

    def __init__(self, model_name="deepseek-chat", dataset_path="/content/",
                 method_k_positive="child", dataset_name="mal_500", debug=False):
        random.seed(0)
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.method_k_positive = method_k_positive
        self.debug = debug

        # Initialize API client
        api_key = get_secret('deepseek')
        if not api_key:
            raise ValueError("DeepSeek API key not found. Set 'deepseek' secret.")

        print(f"Initializing DeepSeek Client for model: {model_name}")
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.deepseek.com"
        )

        # Initialize tokenizer
        hf_token = get_secret('hftoken')
        tokenizer_path = "deepseek-ai/DeepSeek-V3"
        print(f"Loading tokenizer from {tokenizer_path}...")

        self.tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_path,
            token=hf_token,
            trust_remote_code=True
        )

        # Initialize components
        self.matcher = EnglishTextMatcher()  # NEW: Fuzzy matcher
        self.logprobs_extractor = LogProbsExtractor(debug=debug)  # NEW: Softmax extractor

        self.api_stops = API_STOP_SEQUENCES[:4]
        self.full_stops = FULL_STOP_SEQUENCES

        # Load datasets
        print(f"Loading datasets from: {dataset_path}")
        self.data_path_know = self._load_dataset(
            os.path.join(dataset_path, "knowledge.json")
        )
        self.data_path_do_not_know = self._load_dataset(
            os.path.join(dataset_path, "nonknowledge.json")
        )

        # Initialize semantic entropy calculator
        self.semantic_entropy = SemanticEntropyAPI(
            client=self.client,
            tokenizer=self.tokenizer,
            model_name=model_name,
            max_new_tokens=10,
            debug=debug
        )

        # Few-shot examples (English)
        self.list_good_shot = [
            "question: What is the capital of France?\nanswer: Paris\n",
            "question: How many continents are there?\nanswer: Seven\n",
            "question: What is the main component of natural gas?\nanswer: Methane\n",
            "question: Who wrote 'Romeo and Juliet'?\nanswer: Shakespeare\n",
            "question: What is the square root of 64?\nanswer: Eight\n",
        ]

        # Results path
        self.path_results = os.path.join(dataset_path, f"results_{dataset_name}_{method_k_positive}")
        os.makedirs(self.path_results, exist_ok=True)

        print(f"✓ Initialized UncertaintyCalculationAPI V2 (English)")
        print(f"  Model: {model_name}")
        print(f"  Features: Softmax probability, Fuzzy matching, Entropy clipping")
        print(f"  Debug mode: {debug}")

    def _load_dataset(self, data_path, sample_size=20000):
        """Load dataset from JSON file."""
        if not os.path.exists(data_path):
            print(f"Warning: Dataset not found at {data_path}")
            return []

        try:
            with open(data_path, 'r', encoding='utf-8') as f:
                raw_data = json.load(f)
        except Exception as e:
            print(f"Error reading json at {data_path}: {e}")
            return []

        target_list = []
        if isinstance(raw_data, dict):
            if "data" in raw_data:
                target_list = raw_data["data"]
                print(f"Found data nested under 'data' key with {len(target_list)} items.")
            else:
                for key, value in raw_data.items():
                    if isinstance(value, list) and len(value) > 0:
                        print(f"Using list found under key '{key}' with {len(value)} items.")
                        target_list = value
                        break
        elif isinstance(raw_data, list):
            target_list = raw_data

        if not target_list:
            return []

        processed_data = []
        for item in target_list:
            if isinstance(item, list) and len(item) >= 2:
                question_text = item[0]
                if "answer:" in question_text.lower():
                    question_text = question_text.rsplit("answer:", 1)[0].strip()

                processed_data.append({
                    "prompt": question_text,
                    "target_answer": item[1],
                    "ids": item[2] if len(item) > 2 else [],
                    "score": item[3] if len(item) > 3 else 0
                })
            elif isinstance(item, dict):
                processed_data.append(item)

        if len(processed_data) > sample_size:
            processed_data = random.sample(processed_data, sample_size)

        return processed_data

    def get_prompt(self, method, example):
        """Create prompt with appropriate context."""
        idx = random.randint(0, len(self.list_good_shot) - 1)
        single_shot = self.list_good_shot[idx]

        if method == "alice":
            context = ".Alice and Bob are students studying for a test on general world knowledge that requires a vast amount of understanding and knowledge. Alice is the smarter of the two. They are given four questions in total. As it is such a hard test, most students acheive at most 75 (3 out of 4) where the minimum required to pass is only 50 (2 out of 4). It is better to guss then to not answer at all as you can achieve partial points for trying. Bob and Alice only need to pass the test. Alice will ask the questions and Bob will try his best. Give only answer, not sentences.\n"
        elif method == "child":
            context = "  I'm working on a big school project, and there's a lot of information I'm supposed to know. But some of it is kind of hard to understand, and I'm not sure where to start. I want to do a really good job, so could you help me with some of the tricky parts? It would mean a lot!. Give brief answers, not sentences.\n"
        else:
            context = ""

        if isinstance(example, dict):
            question = example.get("prompt", example.get("question", ""))
        elif isinstance(example, list) and len(example) > 0:
            question = example[0]
        else:
            question = str(example)

        question = question.strip()
        if not question.startswith("question:"):
            question = f"question: {question}"

        if not question.endswith("answer:"):
            if question.endswith("?"):
                question = question + "\nanswer:"
            else:
                question = question + "?\nanswer:"

        return context + single_shot + question

    def calculate_probabilities_uncertainty(self, data, with_knowledge=True):
        """Main processing pipeline."""
        if not data:
            print("Error: Input data is empty.")
            return [], []

        # Step 1: Generate all responses
        print("Step 1: Generating responses...")
        all_responses = self._generate_all_responses(data)

        # Step 2: Classify responses using FUZZY MATCHING
        print("Step 2: Classifying with English fuzzy matching...")
        classifications = self._classify_responses(all_responses)

        correct_responses = classifications['correct']
        hallucinated_responses = classifications['hallucination']

        print(f"  Found {len(correct_responses)} correct, {len(hallucinated_responses)} hallucinations")
        print(f"  Match type distribution: {classifications['match_stats']}")

        # Step 3: Extract probabilities with semantic entropy
        print("Step 3: Extracting probabilities with softmax estimation...")
        factuality_stats = self._extract_probabilities(correct_responses, "CORRECT")
        hallucination_stats = self._extract_probabilities(hallucinated_responses, "HALLUCINATION")

        # Save results
        self._save_stats(hallucination_stats, factuality_stats, classifications['match_stats'])

        # Print summary
        self._print_summary(factuality_stats, hallucination_stats)

        return factuality_stats, hallucination_stats

    def _generate_all_responses(self, data):
        """Generate responses with logprobs."""
        all_responses = []

        for i, example in enumerate(tqdm(data, desc="Generating responses")):
            prompt = self.get_prompt(self.method_k_positive, example)
            if not prompt:
                continue

            if isinstance(example, dict):
                answer = example.get("target_answer", example.get("answer", ""))
            elif isinstance(example, list) and len(example) > 1:
                answer = example[1]
            else:
                answer = ""

            try:
                messages = [
                    {
                        "role": "system",
                        "content": (
                            "You must provide ONLY the direct answer, not sentences. "
                            "Use minimum words possible. Never explain. "
                            "Never say 'The answer is' or similar phrases. "
                            "Examples: 'Paris', 'Tom Hardy', 'Pacific Ocean'"
                        )
                    },
                    {"role": "user", "content": prompt}
                ]

                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=messages,
                    temperature=0.01,
                    max_tokens=10,
                    logprobs=True,
                    top_logprobs=20,
                    stop=self.api_stops
                )

                full_response = response.choices[0].message.content.strip() if response.choices[0].message.content else ""

                logprobs_content = None
                if response.choices[0].logprobs:
                    logprobs_content = response.choices[0].logprobs.content

                all_responses.append({
                    'prompt': prompt,
                    'full_response': full_response,
                    'logprobs': logprobs_content,
                    'true_answer': answer,
                    'example_id': i
                })

            except Exception as e:
                print(f"Error generating response {i}: {e}")
                all_responses.append({
                    'prompt': prompt,
                    'full_response': "",
                    'logprobs': None,
                    'true_answer': answer,
                    'example_id': i
                })

        return all_responses

    def _classify_responses(self, all_responses):
        """
        Classify responses using FUZZY MATCHING.

        Threshold: score >= 0.3 → CORRECT
        """
        correct = []
        hallucination = []
        match_stats = defaultdict(int)

        for resp in all_responses:
            score, match_type = self.matcher.calculate_match_score(
                resp['full_response'],
                resp['true_answer']
            )

            resp['match_score'] = score
            resp['match_type'] = match_type
            match_stats[match_type] += 1

            # THRESHOLD: 0.3
            if score >= 0.3:
                correct.append(resp)
            else:
                hallucination.append(resp)

        return {
            'correct': correct,
            'hallucination': hallucination,
            'match_stats': dict(match_stats)
        }

    def _extract_probabilities(self, responses, classification):
        """Extract probabilities using SOFTMAX estimation."""
        stats = []

        for resp in tqdm(responses, desc=f"Processing {classification}"):
            full_answer = resp['full_response']
            logprobs_content = resp['logprobs']

            # USE LogProbsExtractor with softmax
            prob_result = self.logprobs_extractor.extract_all_probabilities(
                logprobs_content,
                max_alternatives=5,
                answer_text=full_answer
            )

            first_token_prob = prob_result["first_token_probability"]
            word_alternatives = prob_result["top_word_alternatives"]
            prob_diff = prob_result["prob_diff_top2"]
            probability_source = prob_result["probability_source"]

            # Get token IDs
            try:
                answer_token_ids = self.tokenizer.encode(full_answer)
            except:
                answer_token_ids = []

            # Calculate semantic entropy
            print(f"\nCalculating semantic entropy for: '{full_answer}'")
            semantic_result, generation_details = self.semantic_entropy.calc_semantic_entropy_per_example(
                resp['prompt'],
                resp['true_answer'],
                temp=1.0,
                num_generations=11
            )

            all_generations = semantic_result.get('all_generations', [])

            # Build output record
            stats.append({
                "prompt": resp['prompt'],
                "full_llm_output": full_answer,
                "true_answer": resp['true_answer'],
                "classification": classification,
                "match_score": float(resp.get('match_score', 0)),
                "match_type": resp.get('match_type', 'unknown'),
                "answer_text": full_answer,
                "answer_token_ids": answer_token_ids,
                # CORRECTED PROBABILITIES
                "first_token_probability": float(first_token_prob),
                "probability_source": probability_source,
                "top_word_alternatives": word_alternatives[:2],
                "prob_diff_top2": float(prob_diff),
                # CLIPPED ENTROPY METRICS
                "semantic_entropy": float(max(0.0, semantic_result.get('semantic_entropy', 0))),
                "regular_entropy": float(max(0.0, semantic_result.get('regular_entropy', 0))),
                "cluster_assignment_entropy": float(max(0.0, semantic_result.get('cluster_assignment_entropy', 0))),
                "all_generations": all_generations,
                "num_generations": len(all_generations),
                "num_semantic_clusters": semantic_result.get('debug_info', {}).get('num_clusters', 1)
            })

        return stats

    def _save_stats(self, hallucination_stats, factuality_stats, match_stats):
        """Save statistics to JSON files."""
        try:
            with open(f"{self.path_results}/hallucination.json", "w", encoding='utf-8') as f:
                json.dump(hallucination_stats, f, ensure_ascii=False, indent=2)
            with open(f"{self.path_results}/factuality.json", "w", encoding='utf-8') as f:
                json.dump(factuality_stats, f, ensure_ascii=False, indent=2)
            with open(f"{self.path_results}/match_analysis.json", "w", encoding='utf-8') as f:
                json.dump(match_stats, f, ensure_ascii=False, indent=2)
            print(f"\n📊 Stats saved to {self.path_results}/")
        except Exception as e:
            print(f"⚠ Error saving stats: {e}")

    def _print_summary(self, factuality_stats, hallucination_stats):
        """Print summary of results."""
        print("\n" + "=" * 80)
        print("RESULTS SUMMARY (with SOFTMAX probability & FUZZY matching)")
        print("=" * 80)

        if factuality_stats:
            probs = [s['first_token_probability'] for s in factuality_stats]
            diffs = [s['prob_diff_top2'] for s in factuality_stats]
            sem_ent = [s['semantic_entropy'] for s in factuality_stats]
            print(f"\nCORRECT ({len(factuality_stats)} samples):")
            print(f"  first_token_probability: mean={np.mean(probs):.4f}, std={np.std(probs):.4f}")
            print(f"  prob_diff_top2:          mean={np.mean(diffs):.4f}, std={np.std(diffs):.4f}")
            print(f"  semantic_entropy:        mean={np.mean(sem_ent):.4f}, std={np.std(sem_ent):.4f}")

        if hallucination_stats:
            probs = [s['first_token_probability'] for s in hallucination_stats]
            diffs = [s['prob_diff_top2'] for s in hallucination_stats]
            sem_ent = [s['semantic_entropy'] for s in hallucination_stats]
            print(f"\nHALLUCINATION ({len(hallucination_stats)} samples):")
            print(f"  first_token_probability: mean={np.mean(probs):.4f}, std={np.std(probs):.4f}")
            print(f"  prob_diff_top2:          mean={np.mean(diffs):.4f}, std={np.std(diffs):.4f}")
            print(f"  semantic_entropy:        mean={np.mean(sem_ent):.4f}, std={np.std(sem_ent):.4f}")

        print("=" * 80)


# ============================================================================
# Usage Example
# ============================================================================
if __name__ == "__main__":
    print("=" * 80)
    print("CORRECTED Uncertainty Calculation API - English V2")
    print("Fixes:")
    print("  1. Softmax-based probability estimation (logprob=0 issue)")
    print("  2. Fuzzy text matching with number normalization")
    print("  3. Entropy clipping to non-negative values")
    print("=" * 80)

    # Example usage:
    # api = UncertaintyCalculationAPI(
    #     model_name="deepseek-chat",
    #     dataset_path="/content/",
    #     method_k_positive="child",
    #     debug=True
    # )
    # factuality, hallucinations = api.calculate_probabilities_uncertainty(api.data_path_know)

Writing /content/Trust_me_Im_wrong/uncertainty_calculation_api.py


In [9]:
!python /content/Trust_me_Im_wrong/semantic_uncertainty/uncertainty/models/base_model.py

In [10]:
!python /content/Trust_me_Im_wrong/calc_semantic_entropy_api.py

2025-12-17 12:10:25.192397: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 12:10:25.200579: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 12:10:25.221846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765973425.273458     818 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765973425.293249     818 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765973425.325927     818 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [11]:
!python Trust_me_Im_wrong/uncertainty_calculation_api.py

2025-12-17 12:11:01.070873: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 12:11:01.077933: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 12:11:01.100550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765973461.142623    1091 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765973461.156655    1091 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765973461.185167    1091 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [12]:
from Trust_me_Im_wrong.uncertainty_calculation_api import UncertaintyCalculationAPI

uncertainty_api = UncertaintyCalculationAPI(
    model_name="deepseek-chat",
    dataset_path="/content/",
    dataset_name="mal_500",
    method_k_positive="alice"
)


Initializing DeepSeek Client for model: deepseek-chat
Loading tokenizer from deepseek-ai/DeepSeek-V3...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Loading datasets from: /content/
Found data nested under 'data' key with 379 items.
Found data nested under 'data' key with 750 items.
Loading English embedding model for semantic clustering...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initialized SemanticEntropyAPI with corrected logprobs handling
✓ Initialized UncertaintyCalculationAPI V2 (English)
  Model: deepseek-chat
  Features: Softmax probability, Fuzzy matching, Entropy clipping
  Debug mode: False


In [ ]:
uncertainty_api.calculate_probabilities_uncertainty(uncertainty_api.data_path_know)

Step 1: Generating responses...


Generating responses:  67%|██████▋   | 254/379 [05:57<02:59,  1.43s/it]